In [ ]:
import google.cloud.bigquery
import tqdm
import pandas as pd
import numpy as np


##Set the project ID - Change to your project ID
##Keep the database name as database itself for smooth processing
##Otherwise you will have to change the database name in the codes down the line - Adviced just to change the project ID to your project
project_id='trilink-472019'
database_name = 'database'
location="US"
# Load the extension
%load_ext google.cloud.bigquery
%load_ext bigquery_magics

d:\Work\Github\TriLink\venv\Lib\site-packages\google\cloud\bigquery\__init__.py:237: FutureWarning: %load_ext google.cloud.bigquery is deprecated. Install bigquery-magics package and use `%load_ext bigquery_magics`, instead.
  warnings.warn(


#### Setup BigQuery

In [2]:
##Authineticate and login to gcloud env
!gcloud auth login
!gcloud config set project $project_id

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=pmsqSOKOhvFNKkBVe5Qmci1BWJdtzz&access_type=offline&code_challenge=Sm3RpmcWf2_JTZSEKJby6C1CAUDUQ8c4sc6SfjVJKEQ&code_challenge_method=S256


You are now logged in as [hazardscarn10@gmail.com].
Your current project is [trilink-472019].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID
Updated property [core/project].


#### Idea

- Business often uses Machine Learning to create models that predicts events of customers/prospects like churn, upsell etc.
- However these models are usually used to create a list of targets and all the targets are treated in the same way or with A/B testing similar cookie cutter content
- This is not efficient as we are losing a lot infomration that the models possess and not using it in the campaigns
- Hence the idea here is to create Xgboost models in bigquery ML for predicting churn for internet and mobile customers of TriLink
- We will use the BigQuery ML.EXPLAIN_PREDICT which not only predicts the probability of event, but also gives the individual contribution of features towards churn
- This helps us creating/identifying exactly what's leading to this customers churn and can be added in as one of the personalization parameter in the automated hyperpersonalized retention email
- Inorder to this let's first create the internet and mobile churn models and the views for the model datasets

#### Mobile Churn Model Creation

##### Create the view for the churn model for mobile


In [ ]:
%%bigquery  --project {project_id}

CREATE OR REPLACE VIEW `database.mobile_churn_data` AS
SELECT 
  c.customer_id,
  
  -- Customer demographics
  c.age,
  c.household_income,
  c.family_size,
  c.home_ownership,
  c.work_from_home_flag,
  c.education_level,
  c.life_stage,
  c.home_square_footage,
  c.property_value,
  c.neighborhood_crime_rate,
  c.neighborhood_income_median,
  c.fiber_availability,
  c.income_bracket,
  
  -- Mobile service features
  m.service_id,
  m.plan_type,
  m.line_count,
  m.monthly_cost,
  m.data_overage_frequency,
  m.contract_type,
  m.family_plan_flag,
  m.device_upgrade_cycle,
  m.mobile_tenure_days,
  m.mobile_churn,
  
  -- Essential calculated features only
  ROUND(c.household_income / c.family_size, 2) AS income_per_person,
  ROUND(m.monthly_cost / m.line_count, 2) AS cost_per_line,
  ROUND(m.mobile_tenure_days / 365.25, 2) AS tenure_years
  
FROM `database.customer_df` c
INNER JOIN `database.mobile_df` m
ON c.customer_id = m.customer_id;

Query is running:   0%|          |

""


In [ ]:
%%bigquery  --project {project_id}

select mobile_churn,count(*) as cnt
FROM `mobile_churn_data`
group by mobile_churn;

Query is running:   0%|          |

Downloading:   0%|          |

,mobile_churn,cnt
0,0,56134
1,1,18261


##### Create Mobile Churn Prediction Model (Boosting Model)

In [ ]:
%%bigquery  --project {project_id}

CREATE OR REPLACE MODEL `database.mobile_churn_predictor`
OPTIONS(
  model_type='BOOSTED_TREE_CLASSIFIER',
  input_label_cols=['mobile_churn'],
  enable_global_explain=TRUE
) AS
SELECT 
  mobile_churn,
  
  -- Demographic features
  age,
  household_income,
  family_size,
  home_square_footage,
  property_value,
  neighborhood_crime_rate,
  neighborhood_income_median,
  
  -- Categorical features
  income_bracket,
  home_ownership,
  education_level,
  life_stage,
  plan_type,
  contract_type,
  
  -- Boolean features
  work_from_home_flag,
  fiber_availability,
  family_plan_flag,
  
  -- Mobile behavior features
  line_count,
  monthly_cost,
  data_overage_frequency,
  device_upgrade_cycle,
  tenure_years,
  income_per_person,
  cost_per_line

FROM `database.mobile_churn_data`
WHERE mobile_churn IS NOT NULL;

Query is running:   0%|          |

""


##### Score the mobile churn model with feature contribution for a customer to demo the scoring

In [3]:
%%bigquery  --project {project_id}
SELECT 
  customer_id,
  predicted_mobile_churn,
  ROUND(probability * 100, 1) AS churn_probability_percent,
  
  -- Risk category
  CASE 
    WHEN probability >= 0.8 THEN 'CRITICAL RISK'
    WHEN probability >= 0.6 THEN 'HIGH RISK'
    WHEN probability >= 0.4 THEN 'MEDIUM RISK'
    WHEN probability >= 0.2 THEN 'LOW RISK'
    ELSE 'VERY LOW RISK'
  END AS risk_category,
  
  -- Customer details
  plan_type,
  monthly_cost,
  contract_type,
  tenure_years,
  data_overage_frequency,
  family_plan_flag,
  
  -- Feature contributions (raw string from BigQuery)
  top_feature_attributions,
  
  -- Prediction details
  ROUND(baseline_prediction_value, 4) AS baseline_prediction,
  ROUND(prediction_value, 4) AS final_prediction,
  approximation_error

FROM ML.EXPLAIN_PREDICT(
  MODEL `database.mobile_churn_predictor`,
  (SELECT * FROM `database.mobile_churn_data` 
   WHERE customer_id = 'C00004244')  -- CHANGE THIS CUSTOMER ID
);

Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,predicted_mobile_churn,churn_probability_percent,risk_category,plan_type,monthly_cost,contract_type,tenure_years,data_overage_frequency,family_plan_flag,top_feature_attributions,baseline_prediction,final_prediction,approximation_error
0,C00004244,0,64.6,HIGH RISK,Limited_2GB,30,Month_to_Month,1.23,4,False,"[{'feature': 'household_income', 'attribution'...",1.091,0.8002,0.0


#### Internet Churn Model

##### Create the view for the churn model for Internet


In [ ]:
%%bigquery  --project {project_id}
-- Create Internet Churn Data View
CREATE OR REPLACE VIEW `database.internet_churn_data` AS 
SELECT 
   c.customer_id,
   
   -- Customer demographics
   c.age,
   c.household_income,
   c.family_size,
   c.home_ownership,
   c.work_from_home_flag,
   c.education_level,
   c.life_stage,
   c.home_square_footage,
   c.property_value,
   c.neighborhood_crime_rate,
   c.neighborhood_income_median,
   c.fiber_availability,
   c.income_bracket,
   
   -- Internet service features
   i.service_id,
   i.plan_tier,
   i.speed_mbps,
   i.monthly_cost,
   i.data_usage_gb,
   i.connected_devices,
   i.contract_type,
   i.speed_complaints,
   i.outage_count,
   i.internet_tenure_days,
   i.internet_churn,
   i.early_termination,
   i.contract_completed_percent,
   
   -- Essential calculated features
   ROUND(c.household_income / c.family_size, 2) AS income_per_person,
   ROUND(i.monthly_cost / i.speed_mbps, 2) AS cost_per_mbps,
   ROUND(i.internet_tenure_days / 365.25, 2) AS tenure_years,
   ROUND(i.data_usage_gb / i.connected_devices, 2) AS data_per_device
   
FROM `database.customer_df` c 
INNER JOIN `database.internet_df` i ON c.customer_id = i.customer_id;


Query is running:   0%|          |

""


In [ ]:
%%bigquery  --project {project_id}

select internet_churn,count(*) as cnt
FROM `database.internet_churn_data`
group by internet_churn;

Query is running:   0%|          |

Downloading:   0%|          |

,internet_churn,cnt
0,0,59009
1,1,13747


##### Create Internet Churn Model - Boosting Model

In [ ]:
%%bigquery  --project {project_id}

-- Create Internet Churn Prediction Model
CREATE OR REPLACE MODEL `database.internet_churn_predictor`
OPTIONS(
   model_type='BOOSTED_TREE_CLASSIFIER',
   input_label_cols=['internet_churn'],
   enable_global_explain=TRUE
) AS 
SELECT 
   internet_churn,
   
   -- Demographic features
   age,
   household_income,
   family_size,
   home_square_footage,
   property_value,
   neighborhood_crime_rate,
   neighborhood_income_median,
   
   -- Categorical features
   income_bracket,
   home_ownership,
   education_level,
   life_stage,
   plan_tier,
   contract_type,
   
   -- Boolean features
   work_from_home_flag,
   fiber_availability,
   early_termination,
   
   -- Internet behavior features
   speed_mbps,
   monthly_cost,
   data_usage_gb,
   connected_devices,
   speed_complaints,
   outage_count,
   tenure_years,
   contract_completed_percent,
   
   -- Calculated features
   income_per_person,
   cost_per_mbps,
   data_per_device
   
FROM `database.internet_churn_data` 
WHERE internet_churn IS NOT NULL;

Query is running:   0%|          |

""


##### Score Internet customer with model with feature contribution

In [4]:
%%bigquery  --project {project_id}
SELECT 
  *,
  ROUND(probability * 100, 1) AS churn_probability_percent,
  
  -- Risk category
  CASE 
    WHEN probability >= 0.8 THEN 'CRITICAL RISK'
    WHEN probability >= 0.6 THEN 'HIGH RISK'
    WHEN probability >= 0.4 THEN 'MEDIUM RISK'
    WHEN probability >= 0.2 THEN 'LOW RISK'
    ELSE 'VERY LOW RISK'
  END AS risk_category,
  
  -- Feature contributions (raw string from BigQuery)
  top_feature_attributions,
  
  -- Prediction details
  ROUND(baseline_prediction_value, 4) AS baseline_prediction,
  ROUND(prediction_value, 4) AS final_prediction,
  approximation_error

FROM ML.EXPLAIN_PREDICT(
  MODEL `database.internet_churn_predictor`,
  (SELECT * FROM `database.internet_churn_data` 
   WHERE customer_id = 'C00000005')
);

Query is running:   0%|          |

Downloading:   0%|          |

,predicted_internet_churn,probability,top_feature_attributions,baseline_prediction_value,prediction_value,approximation_error,customer_id,age,household_income,family_size,...,income_per_person,cost_per_mbps,tenure_years,data_per_device,churn_probability_percent,risk_category,top_feature_attributions_1,baseline_prediction,final_prediction,approximation_error_1
0,0,0.939258,"[{'feature': 'outage_count', 'attribution': 0....",1.601866,1.86923,0.0,C00000005,49,111316,4,...,27829.0,1.72,2.0,3.85,93.9,CRITICAL RISK,"[{'feature': 'outage_count', 'attribution': 0....",1.6019,1.8692,0.0
